In [1]:
!pip install langchain ollama faiss-cpu chromadb jupyter

In [2]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
import ollama
import faiss
import numpy as np
import chromadb

# Define paths
class_files_path = 'class_files'
embedding_model_name = 'mxbai-embed-large'
llm_model_name = 'gemma:2b'

In [3]:
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Load and split files
chunks = []
file_paths = [os.path.join(class_files_path, file) for file in os.listdir(class_files_path) if file.endswith('.rst')]

for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        chunks.extend(text_splitter.split_text(content))

print(f"Total chunks created: {len(chunks)}")

Total chunks created: 289


In [4]:
# Initialize ChromaDB client and collection
client = chromadb.Client()
collection = client.create_collection(name="docs")

# Generate and store embeddings
for i, chunk in enumerate(chunks):
    response = ollama.embeddings(model=embedding_model_name, prompt=chunk)
    embedding = response["embedding"]
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        documents=[chunk]
    )

# Retrieve embeddings for FAISS
embeddings = collection.get()['embeddings']
embeddings_array = np.array(embeddings)

# Print the shape of the embeddings array
print(f"Embeddings array shape: {embeddings_array.shape}")

In [ ]:
# Initialize FAISS index
dimension = embeddings_array.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embeddings_array)

print(f"Number of vectors in the FAISS index: {index.ntotal}")

In [ ]:
# function to handle a query
def handle_user_query(query, index, chunks, top_k=5):
    # Generate embedding for the query
    response = ollama.embeddings(model=embedding_model_name, prompt=query)
    query_embedding = np.array([response["embedding"]])
    
    # Search for top k similar chunks
    _, I = index.search(query_embedding, top_k)
    
    top_chunks = [chunks[i] for i in I[0]]
    combined_text = "\n\n".join(top_chunks)
    
    # Generate response using LLM
    llm_response = ollama.generate(model=llm_model_name, prompt=query + "\n\nContext:\n" + combined_text)
    return llm_response

In [ ]:
# Example query
query = "What is the XRP?"
response = handle_user_query(query, index, chunks)
print(response)

In [ ]:
# Example query
query = "What should I do if my XRP goes crazy while line following?"
response = handle_user_query(query, index, chunks)
print(response)